# Расширение запросов

В данном ноутбуке представлен код для расширения запросов с помощью автоматически сгенерированных синонимов.

In [68]:
import gensim
import pymorphy2
import re
from tqdm.notebook import tqdm
import os

PATH = "/Users/michelle/data/text-relevance-competition-ir-1-ts-fall-2020/"
import requests
import pickle
from joblib import Parallel, delayed

Для этого используем gensim модель, предобученную на корпусе русского языка и википедии.

In [134]:
model = gensim.models.KeyedVectors.load_word2vec_format("/Users/michelle/Downloads/ruwikiruscorpora-superbigrams_skipgram_300_2_2018.vec")

In [135]:
model.init_sims(replace=True)

Модель требует указания части речи слова, для которого запрашиваются синонимы. Я получал из через pymorphy2 и приводил к виду, который принимает gensim.

In [25]:
morph = pymorphy2.MorphAnalyzer()

In [35]:
morph.parse("дуб")[0].tag.POS

'NOUN'

In [63]:
with open(PATH + "queries.numerate.txt", "r") as f:
    queries = f.read().split("\n")
queries = [None] + [item.split("\t")[1] for item in queries if item]

In [143]:
model.most_similar(["выездной_ADJ"])

[('таможенный::проверка_ADJNOUN', 0.5277069807052612),
 ('внеплановый_ADJ', 0.5041821599006653),
 ('выездный_ADJ', 0.5037258863449097),
 ('налоговый::проверка_ADJNOUN', 0.5003116130828857),
 ('новогорск_PROPN', 0.4826815724372864),
 ('туристический::слет_ADJNOUN', 0.47946619987487793),
 ('кубковый::встреча_ADJNOUN', 0.4770793914794922),
 ('предсезоть_ADJ', 0.47122904658317566),
 ('центр::проводить_NOUNVERB', 0.4691962003707886),
 ('межрайоть_ADJ', 0.468818336725235)]

Будем генерировать запрос, в котором одно или несколько слов заменены на синонимы.

In [149]:
class SynGenerator:
    def __init__(self, model, morph):
        self.model = model
        self.morph = morph
        self.tag_pymorphy2gensim = {
            "NOUN": "NOUN",
            "ADJF": "ADJ",
            "ADJS": "ADJ",
            "COMP": "ADV",
            "VERB": "VERB",
            "INFN": "VERB",
            "ADVB": "ADV",
            'INTJ': "INTJ"
        }
        
    def get_synonyms(self, word, num=1):
        pos = self.morph.parse(word)[0].tag.POS
        try:
            pos = self.tag_pymorphy2gensim[pos]
            result = self.model.most_similar([f"{word}_{pos}"], topn=num)
            result = [token  for item in result for token in item[0].split("_")[0].split("::")]
            output = []
            count = 0
            for token in result:
                if token != word:
                    output.append(token)
            return output

        except:
            return [word]

In [150]:
gen = SynGenerator(model, morph)
gen.get_synonyms("виза", 10)

['получение',
 'транзитный',
 'въездный',
 'рабочий',
 'туристический',
 'российский',
 'выездной',
 'въездной',
 'загранпаспорт',
 'паспорт']

Для корректной генерации синонимов обработаем их спеллчекером.

In [65]:
def spellcheck(text):
    payload = {"text": text}
    response = requests.get("https://speller.yandex.net/services/spellservice.json/checkText",
                            params=payload).json()
    
    fixed_text = text
    for item in response:
        fixed_text = fixed_text.replace(item["word"], item["s"][0])
    return fixed_text

In [66]:
fix_queries = [spellcheck(query) for query in tqdm(queries, total=len(queries))]

In [151]:
query_extends = [None]
for query in tqdm(fix_queries[1:], total=len(fix_queries) - 1):
    tokens = query.split(" ")
    extended_tokens = []
    for token in tokens:
        extended_tokens.append(gen.get_synonyms(token, 3))
    query_extends.append(" ".join([" ".join(item) for item in extended_tokens]))

In [155]:
queries_all = [f"{orig}\t{ext}" for orig, ext in zip(fix_queries, query_extends)]

In [156]:
pickle.dump(queries_all, open(os.path.join(PATH, "queries_all_large.pkl"), "wb"))

В целом среди синонимов есть хорошие расширения, однако, например, антонимы тоже часто встречаются в том же контексте, что и оригинальные слова, поэтому генератор иногда выдает и их тоже.

In [132]:
for i in queries_all:
    print(i)

None	None
мультивиза в израиль какие страны можно посетить	мультивиза в израиль какие страны можно посещать
надо ли носить компрес гольфы после операции на голеностопе	надо ли носеть компрес гольфы после операции на голеностопе
жировики на спине можно ли применять пиявки	жировики на спине можно ли применяться пиявки
как прописать просто админку	как прописывать просто админку
хто буде судити суперкубок 2017 між шахтар динамо який відбудеться в одесі	хто буде судити суперкубка 2017 між шахтар динамо який відбудеться в одесі
как доехать от метро мякинино до крокус сити молл	как доезжать от метрополитен мякинино до бегония сити магазинчик
как заменить подшипник ступицы ваз 2121	как заменять шарикоподшипник ступицы жигули 2121
как установить мод сумеречный лес на майнкрафт 1 12	как устанавливать джозайть сумерки леса на майнкрафт 1 12
можно ли ставить горчичники при ларингите	можно ли поставить горчичники при ларингите
самый точный прогноз погоды в балашове	самое-той приблизительный прогноз